# Table of Contents
1. [Data Collection & Preparation](#1-data-collection--preparation)
    1. [Initial Setup & Testing](#11-initial-setup--testing)
        1. [Initialize client](#111-initialize-client)
        2. [Test Connection](#112-test-connection)
    2. [Core Data Extraction Functions](#12-core-data-extraction-functions)
        1. [Day-Ahead Prices](#121-day-ahead-prices)
        2. [Cross-Border Flows](#122-cross-border-flows)
        3. [Load Data](#123-load-data)
    3. [Sample Data Collection Script](#13-sample-data-collection-script)
    4. [Data Processing & Analysis Preparation](#14-data-processing--analysis-preparation)
2. [Analysis & Insights](#2-analysis--insights)

#  1. Data Collection & Preparation

In [4]:
!pip install entsoe-py

## 1.1. Initial Setup & Testing

In [5]:
#%% Import necessary libraries
from entsoe import EntsoePandasClient
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px

### 1.1.1. Initialize client  
The API client is initialized with an API key to authenticate requests

In [6]:
#%% Initialize client
API_KEY = "0f2f4222-713f-49d2-85b7-04ddd5f2dc1c"
client = EntsoePandasClient(api_key=API_KEY)

### 1.1.2. Test Connection  
This test ensures the API key is valid and the connection to ENTSO-E platform is working 
before proceeding with more complex queries.  
A dictionary of country codes is defined mapping country names to their ENTSO-E bidding zone codes
    - Includes Belgium, Netherlands, France, Germany/Luxembourg zone etc.

In [7]:
# ENTSO-E country/bidding zone codes
COUNTRIES = {
    'Belgium': 'BE',
    'Netherlands': 'NL', 
    'France': 'FR',
    'Germany_Luxembourg': 'DE_LU',  # Germany-Luxembourg bidding zone
    'Germany': 'DE',
    'Luxembourg': 'LU'
}

In [8]:
COUNTRIES["Germany_Luxembourg"]

'DE_LU'

In [9]:
def test_api_connection():
    """Test if API connection works"""
    try:
        # Test with a simple query - Belgian day-ahead prices for last week
        start = pd.Timestamp('2024-01-01', tz='Europe/Brussels')
        end = pd.Timestamp('2024-01-07', tz='Europe/Brussels')
        
        prices = client.query_day_ahead_prices('BE', start=start, end=end)
        print("✅ API connection successful!")
        print(f"Retrieved {len(prices)} price points")
        print(prices.head())
        return True
    except Exception as e:
        print(f"❌ API connection failed: {e}")
        return False

# Run test
test_api_connection()

✅ API connection successful!
Retrieved 145 price points
2024-01-01 00:00:00+01:00    0.10
2024-01-01 01:00:00+01:00    0.01
2024-01-01 02:00:00+01:00    0.00
2024-01-01 03:00:00+01:00   -0.01
2024-01-01 04:00:00+01:00   -0.03
Freq: 60min, dtype: float64


True

## 1.2. Core Data Extraction Functions  
The ENTSO-E API provides access to different types of electricity market data. For our European Energy Market Analysis, we need three main data types:  

1. **Day-Ahead Prices** - Electricity prices set one day before delivery
2. **Cross-Border Flows** - Physical electricity flows between countries
3. **Load Data** - Actual electricity consumption by country

Let's extract them.

## 1.2.1. Day-Ahead Prices  
In this subsection, a function is designed to fetch electricity pricing data from the ENTSO-E (European Network of Transmission System Operators for Electricity) platform for multiple European countries.

In [10]:
def get_day_ahead_prices(countries, start_date, end_date):
    """Extract day-ahead electricity prices for multiple countries from ENTSO-E platform.
    
    Args:
        countries (dict): Dictionary mapping country names to their ENTSO-E bidding zone codes.
            Example: {'Belgium': 'BE', 'France': 'FR'}
        start_date (pd.Timestamp): Start date for data extraction, timezone-aware.
        end_date (pd.Timestamp): End date for data extraction, timezone-aware.
    
    Returns:
        dict: Dictionary containing pandas Series of day-ahead prices for each country.
            Keys are country names, values are price series in EUR/MWh.
            Example: {'Belgium': pd.Series(...), 'France': pd.Series(...)}
    
    Raises:
        Exception: If API request fails for any country, error is printed but execution continues.
    
    Note:
        - Prices are in EUR/MWh (Euro per megawatt-hour)
        - Data is timezone-aware, typically in 'Europe/Brussels' timezone
        - Data granularity is hourly
        - Missing data points are possible due to API limitations
    """
    prices_data = {}
    
    for country_name, country_code in countries.items():
        try:
            print(f"Fetching day-ahead prices for {country_name}...")
            prices = client.query_day_ahead_prices(
                country_code, 
                start=start_date, 
                end=end_date
            )
            prices_data[country_name] = prices
            print(f"✅ {country_name}: {len(prices)} records")
        except Exception as e:
            print(f"❌ Failed to get {country_name} data: {e}")
    
    return prices_data

## 1.2.2. Cross-Border Flows  
In this subsection, we focus on extracting cross-border physical electricity flows between European countries. 
These flows represent the actual electricity transferred between countries through interconnectors.

In [11]:
def get_cross_border_flows(from_country, to_country, start_date, end_date):
    """Extract physical electricity flows between two countries from ENTSO-E platform.
    
    Args:
        from_country (str): Name of source country (e.g., 'France')
        to_country (str): Name of destination country (e.g., 'Belgium')
        start_date (pd.Timestamp): Start date for data extraction, timezone-aware
        end_date (pd.Timestamp): End date for data extraction, timezone-aware
    
    Returns:
        pd.Series: Time series of physical flows in MW (positive values indicate flow from source to destination).
                  Returns None if request fails.
                  
    Note:
        - Uses COUNTRIES dictionary to map country names to ENTSO-E codes
        - Positive flow values indicate electricity flowing from source to destination country
        - Flow values are in Megawatts (MW)
        - Data granularity is hourly
    """
    
    try:
        flows = client.query_crossborder_flows(
            COUNTRIES[from_country], COUNTRIES[to_country], 
            start=start_date, end=end_date
        )
        print(f"✅ Flows {from_country}->{to_country}: {len(flows)} records")
        return flows
    except Exception as e:
        print(f"❌Failed to get flows {from_country}->{to_country}: {e}")
        return None

## 1.2.3. Load Data
This subsection deals with extracting actual electricity consumption (load) data.

In [12]:
def get_actual_load(countries, start_date, end_date):
    """Extract actual electricity consumption (load) data for multiple countries from ENTSO-E platform.
    
    Args:
        countries (dict): Dictionary mapping country names to their ENTSO-E bidding zone codes.
            Example: {'Belgium': 'BE', 'France': 'FR'}
        start_date (pd.Timestamp): Start date for data extraction, timezone-aware.
        end_date (pd.Timestamp): End date for data extraction, timezone-aware.
    
    Returns:
        dict: Dictionary containing pandas Series of actual load for each country.
            Keys are country names, values are load series in megawatts (MW).
            Example: {'Belgium': pd.Series(...), 'France': pd.Series(...)}
    
    Raises:
        Exception: If API request fails for any country, error is printed but execution continues.
    
    Note:
        - Load values are in megawatts (MW)
        - Data granularity is 15 minutes
    """
    load_data = {}
    
    for country_name, country_code in countries.items():
        try:
            load = client.query_load(
                country_code, 
                start=start_date, 
                end=end_date
            )
            load_data[country_name] =  load
            print(f"✅ Load data for {country_name}: {len(load)} records")
        except Exception as e:
            print(f"❌ Failed to get load data for {country_name}: {e}")
    
    return load_data

## 1.3. Sample Data Collection Script  
This section is focused on European electricity market data collection and analysis. Instead of manually calling each function multiple times, this script orchestrates the entire data collection process for your European Energy Market Analysis project.

In [13]:
def collect_market_data(start_date='2023-01-01', end_date='2023-12-31'):
    """Collect comprehensive European electricity market data from ENTSO-E platform.
    
    Fetches three types of data:
    1. Day-ahead electricity prices for all countries in COUNTRIES dict
    2. Cross-border physical flows between Belgium and neighboring countries
    3. Actual electricity load (consumption) data for all countries
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format. Defaults to '2023-01-01'
        end_date (str): End date in 'YYYY-MM-DD' format. Defaults to '2023-12-31'
        
    Returns:
        dict: Dictionary containing three data types:
            - 'prices': Day-ahead prices by country (EUR/MWh)
            - 'flows': Cross-border flows between countries (MW)
            - 'loads': Actual electricity consumption by country (MW)
    """
    # Convert dates
    start = pd.Timestamp(start_date, tz='Europe/Brussels')
    end = pd.Timestamp(end_date, tz='Europe/Brussels')
    
    print(f"Collecting data from {start_date} to {end_date}")
    print("=" * 50)
    
    # 1. Day-ahead prices
    print("1. Collecting day-ahead prices...")
    prices = get_day_ahead_prices(COUNTRIES, start, end)
    
    # 2. Cross-border flows (key connections to Belgium)
    print("\n2. Collecting cross-border flows...")
    flows = {}
    # Create connections using country names
    key_connections = [
        ('France', 'Belgium'),  # France to Belgium
        ('Netherlands', 'Belgium'),  # Netherlands to Belgium  
        ('Germany_Luxembourg', 'Belgium'),  # Germany-Luxembourg to Belgium
        ('Belgium', 'France'),  # Belgium to France
        ('Belgium', 'Netherlands'),  # Belgium to Netherlands
        ('Belgium', 'Germany_Luxembourg')   # Belgium to Germany-Luxembourg
    ]
    
    for from_c, to_c in key_connections:
        flow_name = f"{COUNTRIES[from_c]}_to_{COUNTRIES[to_c]}"
        flows[flow_name] = get_cross_border_flows(from_c, to_c, 
                                                  start, end)
    
    # 3. Load data
    print("\n3. Collecting load data...")
    loads = get_actual_load(COUNTRIES, start, end)
    
    return {
        'prices': prices,
        'flows': flows, 
        'loads': loads
    }

# Execute data collection
# market_data = collect_market_data('2023-01-01', '2023-12-31')

# 1.4. Data Processing & Analysis Preparation

In [14]:
def clean_and_prepare_data(market_data, elect_type='prices'):
    """Clean and prepare ENTSO-E market data for analysis.
    
    This function processes raw market data from ENTSO-E API into a standardized DataFrame format.
    It combines data from multiple countries into a single DataFrame with datetime index.
    
    Args:
        market_data (dict): Dictionary containing market data with structure:
            {
                'prices': {country_name: price_series, ...},
                'flows': {connection_name: flow_series, ...},
                'loads': {country_name: load_series, ...}
            }
        data_type (str): Type of data to process. Options: 'prices', 'flows', 'loads'
    
    Returns:
        pd.DataFrame: Cleaned DataFrame with columns:
            - datetime (index): Timezone-aware timestamp
            - value: Data value (price in EUR/MWh, flow in MW, or load in MW)
            - country/connection: Country name or connection name
    """
    
    df = pd.DataFrame()
    
    if isinstance(market_data, dict) and elect_type in market_data:
        data = market_data[elect_type]
    else:
        # Handle case where market_data is directly the data dictionary
        data = market_data
        
    for key, values in data.items():
        if values is not None:
            # Ensure all values are scalars, not arrays
            # Especially some array contains: [actual_load, forecast_load, day_ahead_forecast]
            # Take only the first value (actual load)
            scalar_values = [v[0] if isinstance(v, (np.ndarray, list)) else v for v in values.values]
            temp_df = pd.DataFrame({
             'datetime': values.index,
                'value': scalar_values,
                'entity': key
        })
        df = pd.concat([df, temp_df])
    
    # Reset index and set datetime as index
    df = df.reset_index(drop=True)
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
    df = df.set_index('datetime')
    
    # Rename columns based on data type
    if elect_type == 'prices':
        df = df.rename(columns={'value': 'price(EUR/MWh)', 'entity': 'country'})
    elif elect_type == 'flows':
        df = df.rename(columns={'value': 'flow(MW)', 'entity': 'connection'})
    elif elect_type == 'loads':
        df = df.rename(columns={'value': 'load(MW)', 'entity': 'country'})
    
    return df

In [ ]:
# Electricity type dataframe
prices_df=clean_and_prepare_data(collect_market_data(start_date='2021-01-01', end_date='2023-12-31'))
flows_df=clean_and_prepare_data(collect_market_data(start_date='2021-01-01', end_date='2023-12-31'), 
                                'flows')
loads_df=clean_and_prepare_data(collect_market_data(start_date='2021-01-01', end_date='2023-12-31'), 
                                'loads')


1. Collecting day-ahead prices...
Fetching day-ahead prices for Belgium...
✅ Belgium: 26254 records
Fetching day-ahead prices for Netherlands...
✅ Netherlands: 26254 records
Fetching day-ahead prices for France...
✅ France: 26254 records
Fetching day-ahead prices for Germany_Luxembourg...
✅ Germany_Luxembourg: 26254 records
Fetching day-ahead prices for Germany...
❌ Failed to get Germany data: 
Fetching day-ahead prices for Luxembourg...
❌ Failed to get Luxembourg data: 

2. Collecting cross-border flows...
✅ Flows France->Belgium: 46339 records
✅ Flows Netherlands->Belgium: 26254 records
✅ Flows Germany_Luxembourg->Belgium: 105022 records
✅ Flows Belgium->France: 46339 records
✅ Flows Belgium->Netherlands: 26254 records
✅ Flows Belgium->Germany_Luxembourg: 105022 records

3. Collecting load data...
✅ Load data for Belgium: 105022 records
✅ Load data for Netherlands: 105022 records
✅ Load data for France: 26219 records
✅ Load data for Germany_Luxembourg: 105022 records
✅ Load data for 

Connection Error, retrying in 10 seconds


✅ Flows Belgium->Netherlands: 26254 records
✅ Flows Belgium->Germany_Luxembourg: 105022 records

3. Collecting load data...
✅ Load data for Belgium: 105022 records
✅ Load data for Netherlands: 105022 records
✅ Load data for France: 26219 records
✅ Load data for Germany_Luxembourg: 105022 records
✅ Load data for Germany: 105022 records
✅ Load data for Luxembourg: 105022 records
1. Collecting day-ahead prices...
Fetching day-ahead prices for Belgium...
✅ Belgium: 26254 records
Fetching day-ahead prices for Netherlands...
✅ Netherlands: 26254 records
Fetching day-ahead prices for France...
✅ France: 26254 records
Fetching day-ahead prices for Germany_Luxembourg...
✅ Germany_Luxembourg: 26254 records
Fetching day-ahead prices for Germany...
❌ Failed to get Germany data: 
Fetching day-ahead prices for Luxembourg...
❌ Failed to get Luxembourg data: 

2. Collecting cross-border flows...
✅ Flows France->Belgium: 46339 records
✅ Flows Netherlands->Belgium: 26254 records
✅ Flows Germany_Luxembou

In [29]:
# Filter out 2020 data from existing DataFrame
prices_df = prices_df[prices_df.index.year >= 2021]
flows_df = flows_df[flows_df.index.year >= 2021]
loads_df = loads_df[loads_df.index.year >= 2021]

# Verify the date range
print("Date range check:")
print(f"Prices: {prices_df.index.min()} to {prices_df.index.max()}")
print(f"Flows: {flows_df.index.min()} to {flows_df.index.max()}")
print(f"Loads: {loads_df.index.min()} to {loads_df.index.max()}")

Date range check:
Prices: 2021-01-01 00:00:00+00:00 to 2023-12-30 22:00:00+00:00
Flows: 2021-01-01 00:00:00+00:00 to 2023-12-30 22:45:00+00:00
Loads: 2021-01-01 00:00:00+00:00 to 2023-12-30 22:45:00+00:00


In [16]:
def plot_prices(prices_df):
    """Plot day-ahead electricity prices for multiple countries.
    
    Args:
        prices_df (pd.DataFrame): DataFrame containing price data with columns:
            - datetime (index): Timezone-aware timestamp
            - price: Price in EUR/MWh
            - country: Country name
    """
    plt.figure(figsize=(14, 8))
    sns.lineplot(data=prices_df, x=prices_df.index, y='price', hue='country')
    plt.title('Day-Ahead Electricity Prices by Country')
    plt.xlabel('Date')
    plt.ylabel('Price (EUR/MWh)')
    plt.xticks(rotation=45)
    plt.legend(title='Country')
    plt.grid()
    plt.tight_layout()
    plt.show()


In [17]:
def visualize_prices(price_df):
    """Visualize day-ahead prices using seaborn and plotly"""
    
    # Seaborn line plot
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=price_df, x=price_df.index, y='price', hue='country')
    plt.title('Day-Ahead Electricity Prices by Country')
    plt.xlabel('Date')
    plt.ylabel('Price (EUR/MWh)')
    plt.legend(title='Country')
    plt.grid()
    plt.show()
    
    # Plotly interactive plot
    fig = px.line(price_df.reset_index(), x='datetime', y='price', color='country',
                  title='Day-Ahead Electricity Prices by Country',
                  labels={'datetime': 'Date', 'price': 'Price (EUR/MWh)', 'country': 'Country'})
    fig.show()

# 2. Analysis & Insights

## 2.1. Price volatility analysis
Price volatility refers to the degree of variation in energy prices over a certain period of time—how much and how quickly prices can change.  

In the energy industry, this often means how much the prices of commodities like oil, natural gas, coal, or electricity fluctuate over days, months, or years. These fluctuations can happen daily (short-term) or over longer periods (seasonal, yearly).  
It's pertinent because:
-  It affects businesses, consumers, and governments.
-  It shapes investment strategies and market behavior.

In [25]:
# Visualize prices
visualize_prices(prices_df)
# Visualize flows

,price(EUR/MWh),country
datetime,,
2020-12-31 23:00:00+00:00,50.87,Belgium
2021-01-01 00:00:00+00:00,48.19,Belgium
2021-01-01 01:00:00+00:00,44.68,Belgium
2021-01-01 02:00:00+00:00,42.92,Belgium
2021-01-01 03:00:00+00:00,40.39,Belgium


In [28]:
prices_df.pivot_table(
    index=prices_df.index.year,  # Group by year
    columns='country',
    values='price(EUR/MWh)',
    aggfunc=['mean', 'min', 'max']  # Get statistics per year
)

mean                                                min  \
country      Belgium      France Germany_Luxembourg Netherlands Belgium   
datetime                                                                  
2020       50.870000   50.870000          50.870000   50.870000   50.87   
2021      104.134160  109.174377          96.860229  102.982332  -70.00   
2022      244.549612  275.899632         235.466722  241.936802 -100.00   
2023       97.519091   97.107185          95.421232   96.051651 -120.00   

                                                    max           \
country   France Germany_Luxembourg Netherlands Belgium   France   
datetime                                                           
2020       50.87              50.87       50.87   50.87    50.87   
2021      -66.18             -69.00      -66.18  620.00   620.00   
2022       -1.44             -19.04     -222.36  871.00  2987.78   
2023     -134.94            -500.00     -500.00  330.36   276.12   

                                         
country  Germany_Luxembourg Netherlands  
datetime                                 
2020                  50.87       50.87  
2021                 620.00      620.00  
2022                 871.00      871.00  
2023                 524.27      463.77

In [ ]:
pivot_table = flows_df.pivot_table(index=flows_df.index, columns='connection', values='flow(MW)')
def visualize_flows(flows_df):
    """Visualize cross-border electricity flows using seaborn and plotly"""
    
    # Seaborn heatmap
    plt.figure(figsize=(14, 8))
    sns.heatmap(flows_df.pivot_table(index=flows_df.index, columns='connection', values='flow(MW)'),
                cmap='coolwarm', annot=False)
    plt.title('Cross-Border Electricity Flows (MW)')
    plt.xlabel('Connection')
    plt.ylabel('Date')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Plotly interactive heatmap
    fig = px.imshow(pivot_table.T, 
                    labels=dict(x="Date", y="Connection", color="Flow (MW)"),
                    x=flows_df.index, y=pivot_table.columns,
                    title='Cross-Border Electricity Flows (MW)')
    fig.update_xaxes(tickangle=45)
    fig.show()

In [31]:
 # Pivot to have countries as columns
prices_pivot = prices_df.pivot(columns='country', values='price(EUR/MWh)')
    
# Add year column for annual analysis
prices_pivot['year'] = prices_pivot.index.year

In [33]:
print(f"✅ Data loaded: {len(prices_pivot)} hours across {len(prices_pivot.columns)-1} countries")
print(f"📅 Date range: {prices_pivot.index.min()} to {prices_pivot.index.max()}")
print(f"🌍 Countries: {[col for col in prices_pivot.columns if col != 'year']}")
print(f"📊 Coverage: {prices_pivot['year'].value_counts().sort_index().to_dict()} hours per year")

✅ Data loaded: 26253 hours across 4 countries
📅 Date range: 2021-01-01 00:00:00+00:00 to 2023-12-30 22:00:00+00:00
🌍 Countries: ['Belgium', 'France', 'Germany_Luxembourg', 'Netherlands']
📊 Coverage: {2021: 8759, 2022: 8759, 2023: 8735} hours per year


In [ ]:
# Remove year column from main dataframe but keep for reference
year_data = prices_pivot['year']
prices_pivot = prices_pivot.drop('year', axis=1)